In [2]:
from sklearn.neural_network import MLPRegressor
import mlflow
import os

In [3]:
# Establecer la URI de la base de datos SQLite y configuración de MLflow
os.environ['MLFLOW_TRACKING_URI'] = 'sqlite:///mlruns.db'
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("redes_neuronales")

<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1713175269356, experiment_id='2', last_update_time=1713175269356, lifecycle_stage='active', name='redes_neuronales', tags={'mlflow.sharedViewState.35fcf56d61b8dbb98512f6829913993e41257533dd332e1e8ff07999f73a21ea': '{"searchFilter":"","orderByKey":"attributes.start_time","orderByAsc":false,"startTime":"ALL","lifecycleFilter":"Active","datasetsFilter":[],"modelVersionFilter":"All '
                                                                                            'Runs","selectedColumns":["attributes.`Source`","attributes.`Models`","attributes.`Dataset`"],"runsExpanded":{},"runsPinned":[],"runsHidden":[],"runsHiddenMode":"FIRST_10_RUNS","compareRunCharts":[{"uuid":"1713177259273xowwzda7","type":"BAR","runsCountToCompare":10,"metricSectionId":"171317725927396vp3vfh","deleted":false,"isGenerated":true,"metricKey":"best_mean_test_accuracy"},{"uuid":"1713177259273vnosmd9g","type":"BAR","runsCountToCompare":10,"metri

# Sin Escalado | Con Hiperparámetros

In [20]:
with mlflow.start_run(run_name="evaluación_modelo_sin_escalado_hiperparametros"):
    import preprocesamiento_datos
    from sklearn.metrics import mean_absolute_error
    
    # Obtener los datos preprocesados
    X_train, X_val, X_test, y_train, y_val, y_test, RANDOM_STATE = preprocesamiento_datos.preprocesamiento(True,[], False)

    # Entrenamiento del modelo
    clf = MLPRegressor(random_state=RANDOM_STATE, solver="adam", alpha=0.00001, learning_rate_init=0.0001, hidden_layer_sizes=
(10, 10), batch_size=16)
    
    # Ajustar el modelo con los datos de entrenamiento
    clf.fit(X_train, y_train)
    
    # Predicción en el conjunto de validación
    val_pred = clf.predict(X_val)
    
    # Predicción en el conjunto de prueba
    test_pred = clf.predict(X_test)
    
    # Calcular métricas de evaluación en el conjunto de validación
    mae_val = mean_absolute_error(y_val, val_pred)
    
    # Calcular métricas de evaluación en el conjunto de prueba
    mae_test = mean_absolute_error(y_test, test_pred)
    
    print("Validation MAE:", mae_val)
    print("Test MAE:", mae_test)
    
    # Registro de hiperparámetros y métricas en MLflow
    mlflow.log_param("solver", 'adam')
    mlflow.log_param("alpha", 0.01)
    mlflow.log_param("batch_size", 16)
    mlflow.log_param("learning_rate_init", 0.1)
    mlflow.log_param("hidden_layer_sizes", (185,))
    mlflow.log_metric("val_mae", mae_val)
    mlflow.log_metric("test_mae", mae_test)

Validation MAE: 3075853.932216402
Test MAE: 2960461.587433344


# Hiperparámetros Escalado

In [13]:
with mlflow.start_run(run_name="MODELO FINAL"):
    import preprocesamiento_datos
    from sklearn.metrics import mean_absolute_error, max_error, r2_score
    
    # Obtener los datos preprocesados
    X_train, X_val, X_test, y_train, y_val, y_test, RANDOM_STATE = preprocesamiento_datos.preprocesamiento(False,[], False)
    
    # Entrenamiento del modelo
    clf = MLPRegressor(random_state=RANDOM_STATE, solver="adam", alpha=0.01, learning_rate_init=0.1, 
    batch_size= 16,
    hidden_layer_sizes=(185,))
    
    # Ajustar el modelo con los datos de entrenamiento
    clf.fit(X_train, y_train)
    
    # Predicción en el conjunto de validación
    val_pred = clf.predict(X_val)
    
    # Predicción en el conjunto de prueba
    test_pred = clf.predict(X_test)
    
    # Calcular métricas de evaluación en el conjunto de validación
    mae_val = mean_absolute_error(y_val, val_pred)
    
    # Calcular métricas de evaluación en el conjunto de prueba
    mae_test = mean_absolute_error(y_test, test_pred)
    
    print("Validation MAE:", mae_val)
    print("Test MAE:", mae_test)
    
    # Registro de hiperparámetros y métricas en MLflow
    mlflow.log_param("solver", 'adam')
    mlflow.log_param("alpha", 0.01)
    mlflow.log_param("batch_size", 16)
    mlflow.log_param("learning_rate_init", 0.1)
    mlflow.log_param("hidden_layer_sizes", (185,))
    mlflow.log_metric("val_mae", mae_val)
    mlflow.log_metric("test_mae", mae_test)

Archivo Datos_la_liga_preparados_entrenamiento.parquet guardado en: ../Downloads\Datos_la_liga_preparados_entrenamiento.parquet
Validation MAE: 3004416.477725017
Test MAE: 2629609.357214961


# Default

In [1]:
with mlflow.start_run(run_name="evaluación_modelo_default"):
    import preprocesamiento_datos
    from sklearn.metrics import mean_absolute_error, max_error, r2_score
    
    # Obtener los datos preprocesados
    X_train, X_val, X_test, y_train, y_val, y_test, RANDOM_STATE = preprocesamiento_datos.preprocesamiento(False,[], False)
    
    # Entrenamiento del modelo
    clf = MLPRegressor(random_state=RANDOM_STATE)
    
    # Ajustar el modelo con los datos de entrenamiento
    clf.fit(X_train, y_train)
    
    # Predicción en el conjunto de validación
    val_pred = clf.predict(X_val)
    
    # Predicción en el conjunto de prueba
    test_pred = clf.predict(X_test)
    
    # Calcular métricas de evaluación en el conjunto de validación
    mae_val = mean_absolute_error(y_val, val_pred)
    
    # Calcular métricas de evaluación en el conjunto de prueba
    mae_test = mean_absolute_error(y_test, test_pred)
    
    print("Validation MAE:", mae_val)
    print("Test MAE:", mae_test)
    
    mlflow.log_metric("val_mae", mae_val)
    mlflow.log_metric("test_mae", mae_test)

NameError: name 'mlflow' is not defined

In [10]:
#Completo por precio

from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
import mlflow
import preprocesamiento_datos
import pandas as pd
import numpy as np

# Obtener los datos preprocesados
X_train, X_val, X_test, y_train, y_val, y_test, RANDOM_STATE = preprocesamiento_datos.preprocesamiento(False,[], False)

# Crear el modelo de red neuronal
model = MLPRegressor(random_state=RANDOM_STATE, alpha=0.1, batch_size=64, hidden_layer_sizes=(100, 170, 10), learning_rate_init=0.0001, solver='adam')

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de validación
y_val_pred = model.predict(X_val)

# Calcular el error absoluto medio en el conjunto de validación
val_mae = mean_absolute_error(y_val, y_val_pred)

# Iniciar el registro de la ejecución en MLflow
with mlflow.start_run(run_name="COMPLETO POR PRECIO"):
    # Registrar el error absoluto medio en el conjunto de validación en MLflow
    mlflow.log_metric("val_mae", val_mae)

    # Entrenar el modelo final con todos los datos de entrenamiento y validación
    X_train_final = np.concatenate((X_train, X_val))
    y_train_final = np.concatenate((y_train, y_val))
    model.fit(X_train_final, y_train_final)

    # Realizar predicciones en el conjunto de prueba
    y_test_pred = model.predict(X_test)
    
    # Calcular el error absoluto medio en el conjunto de prueba
    test_mae = mean_absolute_error(y_test, y_test_pred)

    # Registrar el error absoluto medio en el conjunto de prueba en MLflow
    mlflow.log_metric("test_mae", test_mae)
    print("Test MAE:", test_mae)
    
    # Crear DataFrame para comparar predicciones vs. valores reales
    pred_vs_real = pd.DataFrame({'y_test': y_test, 'y_pred_rf': abs(y_test_pred), 'dif': abs(y_test-abs(y_test_pred))})
    
    # Definir los rangos de precios
    rangos_precios = [(0, 1000000), (1000000, 5000000), (5000000, 10000000), (10000000, 20000000), (20000000, 40000000), (40000000, 80000000)]

    # Calcular el error medio absoluto para cada rango de precios
    error_rango = []
    for rango in rangos_precios:
        # Seleccionar las filas donde y_test entra dentro de cada intervalo
        pred_vs_real_rango = pred_vs_real[(pred_vs_real['y_test'] >= rango[0]) & (pred_vs_real['y_test'] < rango[1])]
        # Calcular la media de las diferencias para el intervalo actual
        mean_dif = pred_vs_real_rango['dif'].mean()
        error_rango.append(mean_dif)
        # Imprimir y registrar el error medio absoluto para el intervalo actual
        print(f'Rango de precios {rango[0]/1000000}M-{rango[1]/1000000}M: {mean_dif}')
        mlflow.log_metric(f"error_rango_{rango[0]/1000000}M_{rango[1]/1000000}M", mean_dif)


ModuleNotFoundError: No module named 'preprocesamiento_datos'

In [ ]:
# completo por posiciones

from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
import mlflow
import preprocesamiento_datos
import pandas as pd
import numpy as np

# Obtener los datos preprocesados
X_train_completo, X_val_completo, X_test_completo, y_train_completo, y_val_completo, y_test_completo, RANDOM_STATE = preprocesamiento_datos.preprocesamiento(False,[], False)
# Obtener los datos preprocesados para las características seleccionadas
X_train, X_val, X_test, y_train, y_val, y_test, RANDOM_STATE = preprocesamiento_datos.preprocesamiento(False,[], False)

# Crear el modelo de red neuronal
model = MLPRegressor(random_state=RANDOM_STATE, alpha=0.1, batch_size=64, hidden_layer_sizes=(100, 170, 10), learning_rate_init=0.0001, solver='adam')

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de validación
y_val_pred = model.predict(X_val)

# Calcular el error absoluto medio en el conjunto de validación
val_mae = mean_absolute_error(y_val, y_val_pred)

# Iniciar el registro de la ejecución en MLflow
with mlflow.start_run(run_name="COMPLETO EN POSICIONES"):
    # Registrar el error absoluto medio en el conjunto de validación en MLflow
    mlflow.log_metric("val_mae", val_mae)

    # Entrenar el modelo final con todos los datos de entrenamiento y validación
    X_train_final = np.concatenate((X_train, X_val))
    y_train_final = np.concatenate((y_train, y_val))
    model.fit(X_train_final, y_train_final)

    # Realizar predicciones en el conjunto de prueba
    y_test_pred = model.predict(X_test)
    
    # Calcular el error absoluto medio en el conjunto de prueba
    test_mae = mean_absolute_error(y_test, y_test_pred)

    # Registrar el error absoluto medio en el conjunto de prueba en MLflow
    mlflow.log_metric("test_mae", test_mae)
    print("Test MAE:", test_mae)
    
    # Calcular medias por grupos de posiciones
    li = []
    posiciones = ['Left-Back', 'Centre-Back', 'Defensive Midfield',
       'Centre-Forward','Second Striker','Right Winger', 'Left Winger','Central Midfield', 
       'Attacking Midfield', 'Right-Back',
       'Left Midfield', 'Goalkeeper']
    
    # Crear DataFrame para comparar predicciones vs. valores reales
    pred_vs_real = pd.DataFrame({'y_test': y_test, 'y_pred_rf': y_test_pred, 'dif': abs(y_test-y_test_pred)})
    
    for p in posiciones:
        # Filtrar datos por posición
        solo_posicion = pred_vs_real[X_test_completo['position_' + p] == 1]
        # Calcular el error medio absoluto para esta posición
        mean_dif = np.mean(solo_posicion['dif'])
        li.append(mean_dif)
    
    # Calcular medias de errores para cada posición
    mean_defenders = np.mean(li[0:3])
    mean_strikers = np.mean(li[3:7])
    mean_goalkeepers = li[11]
    mean_midfielders = 4 * test_mae - (mean_defenders + mean_strikers + mean_goalkeepers)
    
    print('Defensas:', mean_defenders)
    print('Delanteros:', mean_strikers)
    print('Mediocampistas:', mean_midfielders)
    print('Porteros:', mean_goalkeepers)
    
    # Guardar las métricas específicas en MLflow
    mlflow.log_metric("mean_error_defenders", mean_defenders)
    mlflow.log_metric("mean_error_strikers", mean_strikers)
    mlflow.log_metric("mean_error_midfielders", mean_midfielders)
    mlflow.log_metric("mean_error_goalkeepers", mean_goalkeepers)
